<a href="https://colab.research.google.com/github/TuragDev/Predicting-Crystal-System-with-ML-DL/blob/main/Crystal_structure_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 3.7 MB/s eta 0:00:00


In [ ]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier  # Replace with the appropriate ANN class
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import joblib
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.preprocessing import StandardScaler
import pandas as pd
from sklearn.model_selection import train_test_split
import pickle
from sklearn.preprocessing import LabelEncoder
import sys
from sklearn.model_selection import GridSearchCV
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from pandas import read_csv
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from skopt import BayesSearchCV

In [ ]:
from google.colab import files
uploaded= files.upload()

Saving magpie_testdataset.csv to magpie_testdataset.csv
Saving magpie_traindataset.csv to magpie_traindataset.csv
Saving Orbital_testdataset.csv to Orbital_testdataset.csv
Saving Orbital_traindataset.csv to Orbital_traindataset.csv
Saving Sine_testdataset.csv to Sine_testdataset.csv
Saving Sine_traindataset.csv to Sine_traindataset.csv
Saving Xrd_testdataset.csv to Xrd_testdataset.csv
Saving Xrd_traindataset.csv to Xrd_traindataset.csv


Magpie

In [5]:
df= pd.read_csv('magpie_traindataset.csv')
df.shape

(584, 134)

In [6]:
X_train=df.iloc[:,1:132]
y_train=df['crystal_system']

y_train

0         triclinic
1         triclinic
2      orthorhombic
3         triclinic
4         triclinic
           ...     
579       triclinic
580      monoclinic
581       triclinic
582       triclinic
583      monoclinic
Name: crystal_system, Length: 584, dtype: object

In [7]:
df=pd.read_csv('magpie_testdataset.csv')
X_test=df.iloc[:,1:132]

y_test=df['crystal_system']

In [8]:
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)

In [9]:
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [14]:
clf = MLPClassifier(max_iter=60, random_state=42)
param_space = {
    'activation': ['relu', 'tanh'],    # Activation functions
    'alpha': (1e-5, 1e-2, 'log-uniform'),  # L2 regularization term
    'learning_rate_init': (1e-4, 1e-1, 'log-uniform'),  # Initial learning rate
    'hidden_layer_sizes': (2, 25)
}

In [15]:
bayes_search = BayesSearchCV(
    clf,
    param_space,
    n_iter=20,  # Number of optimization steps
    cv=5,       # Number of cross-validation folds
    random_state=42,
    n_jobs=-1,  # Use all available CPU cores for parallelization
    verbose=1,  # Print progress information
)

In [16]:
bayes_search.fit(X_train_normalized, y_train_encoded)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  warnings.warn(


BayesSearchCV(cv=5, estimator=MLPClassifier(max_iter=60, random_state=42),
              n_iter=20, n_jobs=-1, random_state=42,
              search_spaces={'activation': ['relu', 'tanh'],
                             'alpha': (1e-05, 0.01, 'log-uniform'),
                             'hidden_layer_sizes': (2, 25),
                             'learning_rate_init': (0.0001, 0.1,
                                                    'log-uniform')},
              verbose=1)

In [17]:
best_classifier = bayes_search.best_estimator_
best_classifier.fit(X_train_normalized, y_train_encoded)
pred = best_classifier.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = best_classifier.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)


print(f"Best hyperparameters: {bayes_search.best_params_}")

Accuracy on Training data:  0.6352739726027398
Accuracy on Test data:  0.6575342465753424
Best hyperparameters: OrderedDict([('activation', 'tanh'), ('alpha', 0.01), ('hidden_layer_sizes', 2), ('learning_rate_init', 0.004120660143164957)])


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  warnings.warn(


XRD

In [31]:
df= pd.read_csv('Xrd_traindataset.csv')
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

In [32]:
df=pd.read_csv('Xrd_testdataset.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

In [33]:
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [34]:
clf = MLPClassifier(max_iter=60, random_state=42)
param_space = {
    'activation': ['relu', 'tanh'],    # Activation functions
    'alpha': (1e-5, 1e-2, 'log-uniform'),  # L2 regularization term
    'learning_rate_init': (1e-4, 1e-1, 'log-uniform'),  # Initial learning rate
    'hidden_layer_sizes': (2, 25)
}

In [35]:
bayes_search = BayesSearchCV(
    clf,
    param_space,
    n_iter=20,  # Number of optimization steps
    cv=5,       # Number of cross-validation folds
    random_state=42,
    n_jobs=-1,  # Use all available CPU cores for parallelization
    verbose=1,  # Print progress information
)

In [37]:
bayes_search.fit(X_train_normalized, y_train_encoded)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  warnings.warn(


BayesSearchCV(cv=5, estimator=MLPClassifier(max_iter=60, random_state=42),
              n_iter=20, n_jobs=-1, random_state=42,
              search_spaces={'activation': ['relu', 'tanh'],
                             'alpha': (1e-05, 0.01, 'log-uniform'),
                             'hidden_layer_sizes': (2, 25),
                             'learning_rate_init': (0.0001, 0.1,
                                                    'log-uniform')},
              verbose=1)

In [38]:
best_classifier = bayes_search.best_estimator_
best_classifier.fit(X_train_normalized, y_train_encoded)
pred = best_classifier.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = best_classifier.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)
print(f"Best hyperparameters: {bayes_search.best_params_}")

Accuracy on Training data:  0.8624161073825504
Accuracy on Test data:  0.6778523489932886
Best hyperparameters: OrderedDict([('activation', 'relu'), ('alpha', 0.00031408302724738723), ('hidden_layer_sizes', 13), ('learning_rate_init', 0.0022116021498465193)])


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  warnings.warn(


Sine descriptor

In [41]:
df= pd.read_csv('Sine_traindataset.csv')
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

In [43]:
df=pd.read_csv('Sine_testdataset.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']

In [44]:
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [53]:
clf = MLPClassifier(max_iter=60, random_state=42)
param_space = {
    'activation': ['relu', 'tanh'],    # Activation functions
    'alpha': (1e-5, 1e-2, 'log-uniform'),  # L2 regularization term
    'learning_rate_init': (1e-4, 1e-1, 'log-uniform'),  # Initial learning rate
    'hidden_layer_sizes': (2, 50)
}

In [54]:
bayes_search = BayesSearchCV(
    clf,
    param_space,
    n_iter=20,  # Number of optimization steps
    cv=5,       # Number of cross-validation folds
    random_state=42,
    n_jobs=-1,  # Use all available CPU cores for parallelization
    verbose=1,  # Print progress information
)

In [55]:
bayes_search.fit(X_train_normalized, y_train_encoded)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  warnings.warn(


BayesSearchCV(cv=5, estimator=MLPClassifier(max_iter=60, random_state=42),
              n_iter=20, n_jobs=-1, random_state=42,
              search_spaces={'activation': ['relu', 'tanh'],
                             'alpha': (1e-05, 0.01, 'log-uniform'),
                             'hidden_layer_sizes': (2, 50),
                             'learning_rate_init': (0.0001, 0.1,
                                                    'log-uniform')},
              verbose=1)

In [56]:
best_classifier = bayes_search.best_estimator_
best_classifier.fit(X_train_normalized, y_train_encoded)
pred = best_classifier.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = best_classifier.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)
print(f"Best hyperparameters: {bayes_search.best_params_}")

Accuracy on Training data:  0.7986577181208053
Accuracy on Test data:  0.2214765100671141
Best hyperparameters: OrderedDict([('activation', 'tanh'), ('alpha', 0.0057547661357822966), ('hidden_layer_sizes', 26), ('learning_rate_init', 0.03258136770160393)])


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  warnings.warn(


Orbital

In [58]:
df= pd.read_csv('Orbital_traindataset.csv')
X_train=df.iloc[:,14:]
y_train=df['crystal_system']

In [59]:
df=pd.read_csv('Orbital_testdataset.csv')
X_test=df.iloc[:,14:]

y_test=df['crystal_system']


In [60]:
scaler = StandardScaler()

# fit scaler on the training data
scaler.fit(X_train)

# apply the scaler to both the training and test data
X_train_normalized = scaler.transform(X_train)

X_test_normalized = scaler.transform(X_test)
label_encoder = LabelEncoder()

# Fit and transform label encoder on target values
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.fit_transform(y_test)

In [61]:
clf = MLPClassifier(max_iter=60, random_state=42)
param_space = {
    'activation': ['relu', 'tanh'],    # Activation functions
    'alpha': (1e-5, 1e-2, 'log-uniform'),  # L2 regularization term
    'learning_rate_init': (1e-4, 1e-1, 'log-uniform'),  # Initial learning rate
    'hidden_layer_sizes': (2, 25)
}

In [62]:
bayes_search = BayesSearchCV(
    clf,
    param_space,
    n_iter=20,  # Number of optimization steps
    cv=5,       # Number of cross-validation folds
    random_state=42,
    n_jobs=-1,  # Use all available CPU cores for parallelization
    verbose=1,  # Print progress information
)

In [63]:
bayes_search.fit(X_train_normalized, y_train_encoded)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fits
Fitting 5 folds for each of 1 candidates, totalling 5 fi

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  warnings.warn(


BayesSearchCV(cv=5, estimator=MLPClassifier(max_iter=60, random_state=42),
              n_iter=20, n_jobs=-1, random_state=42,
              search_spaces={'activation': ['relu', 'tanh'],
                             'alpha': (1e-05, 0.01, 'log-uniform'),
                             'hidden_layer_sizes': (2, 25),
                             'learning_rate_init': (0.0001, 0.1,
                                                    'log-uniform')},
              verbose=1)

In [64]:
best_classifier = bayes_search.best_estimator_
best_classifier.fit(X_train_normalized, y_train_encoded)
pred = best_classifier.predict(X_train_normalized)
training_accuracy= accuracy_score(y_train_encoded,pred)
print("Accuracy on Training data: ",training_accuracy)
pred = best_classifier.predict(X_test_normalized)
test_accuracy= accuracy_score(y_test_encoded,pred)
print("Accuracy on Test data: ",test_accuracy)
print(f"Best hyperparameters: {bayes_search.best_params_}")

Accuracy on Training data:  0.6890756302521008
Accuracy on Test data:  0.6644295302013423
Best hyperparameters: OrderedDict([('activation', 'tanh'), ('alpha', 0.004466278825224283), ('hidden_layer_sizes', 9), ('learning_rate_init', 0.07139564472744651)])


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (60) reached and the optimization hasn't converged yet.
  warnings.warn(
